In [7]:
#include "../include/lamppp/tensor/cuda/offset_util.cuh"

In file included from input_line_14:1:
./../include/lamppp/tensor/cuda/offset_util.cuh:3:10: fatal error: 'cuda_runtime.h' file not found
#include <cuda_runtime.h>
         ^~~~~~~~~~~~~~~~


Interpreter Error: 